# Convert the CO2 Manipulator to Python

## Units

### Inputs
- temperature [degrees C]
- salinity [PSU]
- pCO2 [microATMS?] range 150-750 https://tos.org/oceanography/assets/docs/14-4_feely.pdf
- Alkalinity [1 ppm = 1 mg/L]
- TCO2/DIC (Total Dissolved Inorganic Carbon: CO2, HCO3-, and CO32) [umol/L = uM] 20 - 5000 uM
- pH [unitless ratio] 7.5-8.5
- Omega_aragonite (Aragonite saturation state) [unitless ratio] 1.1-4.2

### Outputs
- pCO2
- TCO2/DIC
- CO2aq
- CO3
- HCO3
- pH
- Omega_A
- Omega_B
- Alkalinity

In [1]:
import numpy as np
import math
import gsw

In [2]:
# Import the carbonate chemistry module
from carbonate_chemistry import (
    calculate_carbonate_chemistry_pco2_tco2,
    calculate_alkalinity,
    calculate_ph,
    calculate_omega_aragonite,
    calculate_omega_calcite
)

import carbonate_chemistry as cc

In [3]:
# Thermodynamic constants (equilibrium constants)
Kca = 3.8261e-7 
Kar = 5.48417e-7
Kh = 0.0410032
K1 = 1.04246e-6
K2 = 5.93885e-10
Kb = 1.67762E-9
Kw = 1.82055e-14
Kb12 = 0.0674236
K1K2quotient = K1/K2
print(K1K2quotient)
K1K2product = K1*K2
Kw12 = 7.31679e-7

# Ancilliary constants ()
density = 1.02244
CO2aq = 13.0304
Hplus = 1

1755.323000244155


In [4]:
# test the total boron calculation
salinity = 31
total_boron = cc.calculate_total_boron(salinity)
print( "Labview total boron = 375.72 umol/kg at 35 PSU")
print( f"Python total boron = {total_boron:.3f} umol/kg at 35 PSU" ) 
print( f"Internet total boron = {salinity*432.6/35:.3f} umol/kg at 35 PSU" ) # value from the internet

Labview total boron = 375.72 umol/kg at 35 PSU
Python total boron = 375.720 umol/kg at 35 PSU
Internet total boron = 383.160 umol/kg at 35 PSU


In [5]:
# test the calcium calculation
salinity = 31
calcium = cc.calculate_calcium(salinity)
print( "Labview calcium = 9.41429 ")
print( f"Python calcium = {calcium:.5f} " ) 

Labview calcium = 9.41429 
Python calcium = 9.41429 


## Test case for pCO2 and TCO2/DIC

In [6]:
# Let's do a test calculation with pCO2 and TCO2/DIC
## DEFINE THE INPUTS
calculation_pair = "pCO2, TCO2/DIC"
temperature = 13.2
salinity = 28.8
pCO2 = 317.79
# alkalinity calculated here
TCO2 = 1782
# pH calculated here
# Omega_aragonite not used or calcualted here
print( f"*** Calculation pair: {calculation_pair} ***")

*** Calculation pair: pCO2, TCO2/DIC ***


In [7]:
# Thermodynamic constants (equilibrium constants)
Kca = 3.8261e-7 
Kar = 5.48417e-7
Kh = 0.0410032
K1 = 1.04246e-6
K2 = 5.93885e-10
Kb = 1.67762E-9
Kw = 1.82055e-14
Kb12 = 0.0674236
K1K2quotient = K1/K2
print(K1K2quotient)
K1K2product = K1*K2
Kw12 = 7.31679e-7

# Ancilliary constants ()
density = 1.02244
CO2aq = 13.0304
Hplus = 1

1755.323000244155


In [8]:
# CALCULATE ANCILLIARY CONSTANTS
calcium = cc.calculate_calcium(salinity) # preliminary calculation used for all cases
print( f"Calcium = {calcium:.5f}")

total_boron = cc.calculate_total_boron(salinity) # preliminary calculation used for all cases
print( f"Total Boron = {total_boron:.3f}")

Calcium = 8.81714
Total Boron = 349.056


In [9]:
# CALCULATE GROUPED CONSTANTS
print( "*** GROUPED CONSTANTS ***")
const_A = K1K2quotient**0.5
print( f"const_A = {const_A:.4f}")
const_B = CO2aq**0.5
print( f"const_B = {const_B:.5f}")
# const_C = 1
# const_D = 1
# const_E = 1
# const_F = 1
const_G = TCO2 - CO2aq
print( f"const_G = {const_G:.2f}")
const_H = const_A*const_B
print( f"const_H = {const_H:.3f}")
# const_J = 1


*** GROUPED CONSTANTS ***
const_A = 41.8966
const_B = 3.60976
const_G = 1768.97
const_H = 151.237


In [10]:
# next step of the calculation
print("*** POYNOMIAL COEFFICIENTS ***")
# coefficients = np.array([const_G**2, -1*(const_H**2 + 2*const_G), 1] )
coefficients = np.array([1, -1*(const_H**2 + 2*const_G), const_G**2] ) # reverse input in python as compared to LabView
print(coefficients[::-1])

*** POYNOMIAL COEFFICIENTS ***
[ 3.12925345e+06 -2.64105000e+04  1.00000000e+00]


In [11]:
## NEXT BOX OF THE CALCULATION
print("*** ROOTS ***")
roots = np.roots(coefficients)

# Separate real and imaginary roots
real_roots = []
imaginary_roots = []

for root in roots:
    if np.isclose(root.imag, 0):  # Check if the imaginary part is close to zero
        real_roots.append(root.real)
    else:
        imaginary_roots.append(root)

print(f"Polynomial Coefficients: {coefficients}")
print(f"All Roots: {roots}")
print(f"Real Roots: {real_roots}")
print(f"Imaginary Roots: {imaginary_roots}")

# Loop through all the roots to find valid CO3
print("\n*** ROOT EVALUATION LOOP ***")
TCO2check = 0.5*TCO2
print(f"TCO2 check = {TCO2check}")

CO3 = None

for root_ind in range(len(roots)):
    print(f"\n--- Checking root index {root_ind} ---")
    realroot = roots[root_ind].real
    imagroot = roots[root_ind].imag

    print("Real root: {realroot}")
    print("Imag root: {imagroot}")

    # logic checks for this root
    bool0 = realroot <= TCO2check
    bool1 = realroot >= 0
    # bool2 = imagroot == 0
    bool2 = np.isclose(imagroot, 0)

    print("Real root <= 0.5* TCO2: {bool0}")
    print("Real root >= 0: {bool1}")
    print("Imaginary root ~=0: {bool2}")

    # check if all conditions are met
    if bool0 and bool1 and bool2: 
        CO3 = realroot
        print(f"*** VALID ROOT FOUND ***")
        print(f"CO3 = {CO3}")
        break
    else: 
        print("Root invalid - continuing search")

if CO3 is None: 
    print("\n*** SOLVER FAILED - NO VALID ROOT ROUND***")
else: 
    print(f"\n*** FINAL RESULT ***")
    print(f"Selected CO3 = {CO3}")


*** ROOTS ***
Polynomial Coefficients: [ 1.00000000e+00 -2.64105000e+04  3.12925345e+06]
All Roots: [26291.478437     119.02158539]
Real Roots: [26291.478436995018, 119.0215853864252]
Imaginary Roots: []

*** ROOT EVALUATION LOOP ***
TCO2 check = 891.0

--- Checking root index 0 ---
Real root: {realroot}
Imag root: {imagroot}
Real root <= 0.5* TCO2: {bool0}
Real root >= 0: {bool1}
Imaginary root ~=0: {bool2}
Root invalid - continuing search

--- Checking root index 1 ---
Real root: {realroot}
Imag root: {imagroot}
Real root <= 0.5* TCO2: {bool0}
Real root >= 0: {bool1}
Imaginary root ~=0: {bool2}
*** VALID ROOT FOUND ***
CO3 = 119.0215853864252

*** FINAL RESULT ***
Selected CO3 = 119.0215853864252


In [12]:
# Calculate HCO3
HCO3 = const_A*((CO2aq*CO3)**0.5)
print(f"HCO3 = {HCO3}")

HCO3 = 1649.9480146135745


In [13]:
# calculate H+ 
Hplus = (CO2aq/CO3*K1K2product)**0.5
# calculate pH
pH = -1*math.log10(Hplus)
print(f"pH = {pH}")

pH = 8.084453113590905


In [14]:
# calculate Omega_A
Omega_A = (CO3*calcium*1000)/(Kar*1e12)
print(f"Omega_A = {Omega_A}")

Omega_A = 1.913562711286736


In [15]:
# Calculate Omega_C
Omega_C = (CO3*calcium*1000)/(Kca*1e12)
print(f"Omega_C = {Omega_C}")


Omega_C = 2.7428198986846604


In [16]:
# calculate alkalinity
OH = 1e6*Kw/(Hplus)
print(f"OH- = {OH}")
Bminus = (Kb*total_boron)/(Hplus+Kb)
print(f"B- = {Bminus}")
alkalinity = 2*CO3 + HCO3 + OH + Bminus-Hplus
print(f"alkalinity = {alkalinity}")

OH- = 2.211341032751536
B- = 59.087716467249095
alkalinity = 1949.2902428781927


In [17]:
# Summarize Output
print(f"*** {calculation_pair} SUMMARY ***")
print(f"CO3 = {CO3:.3f}")
print(f"HCO3 = {HCO3:.2f}")
print(f"CO2aq = {CO2aq:.4f}")
print(f"pCO2 = {pCO2:.2f}")
print(f"pH = {pH:.5f}")
print(f"Omega_A = {Omega_A:.5f}")
print(f"Omega_C = {Omega_C:.5f}")
print(f"alkalinity = {alkalinity:.2f}")
print(f"TCO2 = { TCO2}")

*** pCO2, TCO2/DIC SUMMARY ***
CO3 = 119.022
HCO3 = 1649.95
CO2aq = 13.0304
pCO2 = 317.79
pH = 8.08445
Omega_A = 1.91356
Omega_C = 2.74282
alkalinity = 1949.29
TCO2 = 1782


In [18]:
# do the same with the imported function
result = calculate_carbonate_chemistry_pco2_tco2(temperature, salinity, pCO2, TCO2)
# Summarize Output
print(f"*** {calculation_pair} SUMMARY ***")
print(f"CO3 = {result['co3']:.3f}")
print(f"HCO3 = {result['hco3']:.2f}")
print(f"CO2aq = {result['co2aq']:.4f}")
print(f"pCO2 = {result['pco2']:.2f}")
print(f"pH = {result['ph']:.5f}")
print(f"Omega_A = {result['omega_aragonite']:.5f}")
print(f"Omega_C = {result['omega_calcite']:.5f}")
print(f"alkalinity = {result['alkalinity']:.2f}")
print(f"TCO2 = { result['tco2']}")

*** pCO2, TCO2/DIC SUMMARY ***
CO3 = 119.022
HCO3 = 1649.95
CO2aq = 13.0304
pCO2 = 317.79
pH = 8.08445
Omega_A = 1.91356
Omega_C = 2.74282
alkalinity = 1949.29
TCO2 = 1782


# Test case for pCO2 and Alkalinity

In [19]:
# Let's do a test calculation with pCO2 and Alkalinity
## DEFINE THE INPUTS
calculation_pair = "pCO2, Alkalinity"
temperature = 6
salinity = 28
pCO2 = 450
alkalinity = 1298

## DEFINE THE OUTPUTS
# TCO2 calculated here
# pH calculated here
# Omega_aragonite not used or calculated
# print( f"*** Calculation pair: {calculation_pair} ***")

In [20]:
# CALCULATE ANCILLIARY CONSTANTS
calcium = cc.calculate_calcium(salinity) # preliminary calculation used for all cases
print( f"Calcium = {calcium:.5f}")

total_boron = cc.calculate_total_boron(salinity) # preliminary calculation used for all cases
print( f"Total Boron = {total_boron:.3f}")

Calcium = 8.60000
Total Boron = 339.360


In [21]:
# Thermodynamic constants (equilibrium constants)
Kca = 3.48485e-7 # calculated below
Kar = 5.40578e-7 # calculated below for CaCO3
Kh = 0.0524131 # Calculated below with WEISS
K1 = 8.56732e-7 # Calcualted below with Millero 2010
K2 = 4.41007e-10 # Calculated below with Millero 2010
Kb = 1.34303e-9 # Calculated below with Dickson1990
Kw = 8.64256e-15 # calculated below with Millero 1995
Kb12 = 0.0690939
K1K2quotient = K1/K2
K1K2product = K1*K2
Kw12 = 4.44628e-7

# Ancilliary constants (CALCULATED ELSEWHERE)
density = 1.02287 # [g/cm³] calculated below with GSW
CO2aq = 23.5859
Hplus = 1

In [22]:
# CALCULATE SEAWATER DENSITY and compare against Calculate_SW_Density
pressure = 0 # assume the measurement is at sea level 
# calculate absolute salinity
absolute_salinity = gsw.SA_from_SP(salinity, pressure, 0, 0) # 
# calculate conservative temperature
conservative_temperature = gsw.CT_from_t(absolute_salinity, temperature, pressure)
# calculate density
density_gsw = gsw.rho(absolute_salinity, conservative_temperature, pressure)

print(f"*** SEAWATER DENSITY CALCULATION ***")
print(f"Practical Salinity (SP): {salinity} PSU")
print(f"Absolute Salinity (SA): {absolute_salinity:.3f} g/kg") # round to 3 decimal places (uncertainty is 0.007 g/kg)
print(f"In-situ Temperature: {temperature}°C")
print(f"Conservative Temperature (CT): {conservative_temperature:.3f}°C") # round to 3 decimal places ()
print(f"Seawater Density (GSW): {density_gsw:.2f} kg/m³")
print(f"Previous fixed density: {density*1000:.2f} kg/m³")
print(f"Difference: {abs(density_gsw - density*1000):.5f} kg/m³")
# ALL GROUPED CONSTANTS BELOW ARE VITUALLY THE SAME WHEN DENSITY IS CALCULATED WITH THE GIBBS SEAWATER TOOLBOX

print(cc.seawater_density(salinity, temperature))

*** SEAWATER DENSITY CALCULATION ***
Practical Salinity (SP): 28 PSU
Absolute Salinity (SA): 28.132 g/kg
In-situ Temperature: 6°C
Conservative Temperature (CT): 6.080°C
Seawater Density (GSW): 1022.03 kg/m³
Previous fixed density: 1022.87 kg/m³
Difference: 0.84233 kg/m³
1022.0276734640694


In [23]:
# CALCULATE Kca and Kar following CaCO3_Sol.vi
TK = temperature + 273.15

var1 = (171.9065+(0.077993*TK))-2839.319/TK - (71.595*np.log10(TK)) - ((-0.77712+TK*0.0028426 +178.34/TK)*np.sqrt(salinity)) + (0.07711*salinity) - 0.0041249*(salinity**1.5) - 0.02
Kca_k = 10**(-1*var1)

var2 = var1+0.0385-63.974/TK
Kar_k = 10**(-1*var2)

print(f"*** CaCO3 CALCULATION ***")
print(f"Kca. : {Kca}")
print(f"Kca_k: {Kca_k:.6e}")
print(f".      {cc.calculate_Kca(salinity, temperature):5e}")
print(f"Kar. : {Kar}")
print(f"Kar_k: {Kar_k:.5e}")
print(f".      {cc.calculate_Kar(salinity, temperature):.5e}")


*** CaCO3 CALCULATION ***
Kca. : 3.48485e-07
Kca_k: 3.484850e-07
.      3.484850e-07
Kar. : 5.40578e-07
Kar_k: 5.40578e-07
.      5.40578e-07


In [24]:
# Calculate Weiss CO2 solubility (Kh) following Weiss_Kh.vi
var1 = 9345.17/TK - 167.8108 + 23.3585*np.log(TK) +salinity*(4.7036e-7*TK**2 + 0.023517 - 0.00023656*TK)
Kh_k = np.exp(var1)
print(f"*** WEISS CALCULATION ***")
print(f"Kh. : {Kh}")
print(f"Kh_k: {Kh_k:.16f}")
print(f".     {cc.calculate_Kh(salinity, temperature)}")

*** WEISS CALCULATION ***
Kh. : 0.0524131
Kh_k: 0.0524130529478798
.     0.0524130529478798


In [25]:
# calculate K1 and K2 following Millero_K1K2.vi that is based on Millero 2010 K1 and K2
var0 = 19.568224*np.log(TK) + 6320.813/TK - 126.34048
var1 = 13.40511173181*np.sqrt(salinity) + 0.03184972750547*salinity - 5.218336451311e-5*salinity**2
var2 = (-531.0949512384*np.sqrt(salinity) + -5.778928418011*salinity)/TK
var3 = -2.066275370119*np.log(TK)*np.sqrt(salinity)

K1_k = 10**(-1*(var0+var1+var2+var3))

print(f"*** MILLERO 2010 K1 CALCULATION ***")
print(f"K1. : {K1}")
print(f"K1_k: {K1_k:.5e}")
print(f"K1  : {cc.calculate_K1(salinity, temperature):.5e}")

var0 = 14.613358*np.log(TK) + 5143.692/TK - 90.18333
var1 = 21.57241969749*np.sqrt(salinity) + 0.1212374508709*salinity - 0.0003714066864794*salinity**2
var2 = (-798.2916387922*np.sqrt(salinity) + -18.95099792607*salinity)/TK
var3 = -3.402872930641*np.sqrt(salinity)*np.log(TK)

K2_k = 10**(-1*(var0+var1+var2+var3))

print(f"*** MILLERO 2010 K2 CALCULATION ***")
print(f"K2  : {K2}")
print(f"K2_k: {K2_k:.5e}")
print(f"K2_k: {cc.calculate_K2(salinity, temperature):.5e}")

*** MILLERO 2010 K1 CALCULATION ***
K1. : 8.56732e-07
K1_k: 8.56732e-07
K1  : 8.56732e-07
*** MILLERO 2010 K2 CALCULATION ***
K2  : 4.41007e-10
K2_k: 4.41007e-10
K2_k: 4.41007e-10


In [26]:
# calculate Kb using Dickson's 1990 Kb
var0 = (-8966.9 - 2890.53*np.sqrt(salinity) - 77.942*salinity +1.728*salinity**1.5 - 0.0996*salinity**2)/TK
var1 = 148.0248 + 137.1942*np.sqrt(salinity) + 1.62142*salinity
var2 = (-24.4344 + -25.085*np.sqrt(salinity) + -0.2474*salinity)*np.log(TK)
var3 = 0.053105*np.sqrt(salinity)*TK

Kb_k = np.exp(var0+var1+var2+var3)

print(f"*** DICKSON 1990 Kb CALCULATION ***")
print(f"Kb  : {Kb}")
print(f"Kb_k: {Kb_k:.5e}")
print(f".     {cc.calculate_Kb(salinity, temperature):.5e}")

*** DICKSON 1990 Kb CALCULATION ***
Kb  : 1.34303e-09
Kb_k: 1.34303e-09
.     1.34303e-09


In [27]:
# calclaute Kw using Millero's 1995 pHt Kw
var0 = 148.9802 - 13847.26/TK
var1 = 23.6521*np.log(TK)
var2 = (118.67/TK + 1.0495*np.log(TK) - 5.977)*np.sqrt(salinity)
var3 = 0.01615*salinity

Kw_k = 10**((var0-var1+var2-var3)/np.log(10))

print(f"*** MILLERO'S 1995 Kw CALCULATION ***")
print(f"Kw  : {Kw}")
print(f"Kw_k: {Kw_k:0.5e}")
print(f".     {cc.calculate_Kw(salinity, temperature):.5e}")

*** MILLERO'S 1995 Kw CALCULATION ***
Kw  : 8.64256e-15
Kw_k: 8.64256e-15
.     8.64256e-15


In [28]:
# Calculate Kb12
Kb12_k = Kb/np.sqrt(K1*K2)

print(f"*** BURKE's Kb12 CALCULATION ***")
print(f"Kb12  : {Kb12}")
print(f"Kb12_k: {Kb12_k:0.7f}")
print(f"        {cc.calculate_Kb12(Kb, K1, K2)}")

*** BURKE's Kb12 CALCULATION ***
Kb12  : 0.0690939
Kb12_k: 0.0690940
        0.06909402265384235


In [29]:
# Calcualte Kw12
Kw12_k = Kw/np.sqrt(K1*K2)

print(f"*** BURKE's Kw12 CALCULATION ***")
print(f"Kw12  : {Kw12}")
print(f"Kw12_k: {Kw12_k:0.5e}")
print(f"        {cc.calculate_Kw12(Kw, K1, K2)}")


*** BURKE's Kw12 CALCULATION ***
Kw12  : 4.44628e-07
Kw12_k: 4.44628e-07
        4.4462836751762187e-07


In [30]:
# CALCULATE GROUPED CONSTANTS
print( "*** GROUPED CONSTANTS ***")
const_A = K1K2quotient**0.5
print( f"const_A = {const_A:.4f}")
const_B = np.sqrt(CO2aq)
print( f"const_B = {const_B:.5f}")
const_C = total_boron
print( f"const_C = {const_C:.4f}")
const_D = np.sqrt(CO2aq)/Kb12
print( f"const_D = {const_D:.4f}")
const_E = Kw12/np.sqrt(CO2aq)
print( f"const_E = {const_E:0.5e}")
const_F = np.sqrt(CO2aq)*np.sqrt(K1K2product)
print( f"const_F = {const_F:.5e}")
# const_G = TCO2 - CO2aq
const_G = -1*const_F*const_D
print( f"const_G = {const_G:.5e}")
# const_H = const_A*const_B
const_H = -1*(const_D*alkalinity + const_F)
print( f"const_H = {const_H:.1f}")
const_I = (const_C+const_D*(const_E+const_A*const_B)) - alkalinity
print( f"const_I = {const_I:.0f}")
const_J = (const_D*2)+(const_A*const_B + const_E)
print( f"const_J = {const_J:.3f}")


*** GROUPED CONSTANTS ***
const_A = 44.0758
const_B = 4.85653
const_C = 339.3600
const_D = 70.2889
const_E = 9.15526e-08
const_F = 9.43999e-08
const_G = -6.63526e-06
const_H = -91234.9
const_I = 14087
const_J = 354.633


In [31]:
# Combine the grouped constants?
var0 = const_G
var1 = const_H
var2 = const_I
var3 = const_J
var4 = 2


In [32]:
# next step of the calculation
print("*** POYNOMIAL COEFFICIENTS ***")
# coefficients = np.array([var0, var1, var2, var3, var4])
coefficients = np.array([var4, var3, var2, var1, var0] ) # reverse input in python as compared to LabView
print(coefficients[::-1])

*** POYNOMIAL COEFFICIENTS ***
[-6.63526056e-06 -9.12349444e+04  1.40870635e+04  3.54633023e+02
  2.00000000e+00]


In [33]:
## NEXT BOX OF THE CALCULATION
print("*** ROOTS ***")
roots = np.roots(coefficients)

# Separate real and imaginary roots
real_roots = []
imaginary_roots = []

for root in roots:
    if np.isclose(root.imag, 0):  # Check if the imaginary part is close to zero
        real_roots.append(root.real)
    else:
        imaginary_roots.append(root)

print(f"Polynomial Coefficients: {coefficients}")
print(f"All Roots: {roots}")
print(f"Real Roots: {real_roots}")
print(f"Imaginary Roots: {imaginary_roots}")

# Loop thought lal the roots to find valid CO3
print("*** ROOT EVALUATION LOOP ***")
alkalinitycheck = 0.7*np.sqrt(alkalinity)
print("Alkalinity check = {alkalinitycheck}")

CO3 = None
for root_ind in range(len(roots)):
    print(f"\n--- Checking root index {root_ind} ---")
    realroot = roots[root_ind].real
    imagroot = roots[root_ind].imag

    print(f"Real Root: {realroot}")
    print(f"Imaginary Root: {imagroot}")

    # Logic checks for this root
    bool0 = realroot <= alkalinitycheck
    bool1 = realroot >= 0
    bool2 = abs(imagroot) < 1e-10  # Essentially zero imaginary part

    print(f"Real root <= 0.7*sqrt(alkalinity): {bool0}")
    print(f"Real root >= 0: {bool1}")
    print(f"Imaginary root ~= 0: {bool2}")

    # Check if all conditions are met
    if bool0 and bool1 and bool2: 
        CO3 = realroot
        print(f"*** VALID ROOT FOUND ***")
        print(f"CO3 = {CO3}")
        break 
    else: 
        print("Root invalid - continuing search")


if CO3 is None: 
    print("\n*** SOLVER FAILED - NO VALID ROOT FOUND ***")
else:
    print(f"\n*** FINAL RESULT ***")
    print(f"Selected CO3 = {CO3}")

*** ROOTS ***
Polynomial Coefficients: [ 2.00000000e+00  3.54633023e+02  1.40870635e+04 -9.12349444e+04
 -6.63526056e-06]
All Roots: [-1.08573075e+02 -7.43913331e+01  5.64789658e+00 -7.27271836e-11]
Real Roots: [-108.57307524772102, -74.39133305581599, 5.647896584276684, -7.272718362868314e-11]
Imaginary Roots: []
*** ROOT EVALUATION LOOP ***
Alkalinity check = {alkalinitycheck}

--- Checking root index 0 ---
Real Root: -108.57307524772102
Imaginary Root: 0.0
Real root <= 0.7*sqrt(alkalinity): True
Real root >= 0: False
Imaginary root ~= 0: True
Root invalid - continuing search

--- Checking root index 1 ---
Real Root: -74.39133305581599
Imaginary Root: 0.0
Real root <= 0.7*sqrt(alkalinity): True
Real root >= 0: False
Imaginary root ~= 0: True
Root invalid - continuing search

--- Checking root index 2 ---
Real Root: 5.647896584276684
Imaginary Root: 0.0
Real root <= 0.7*sqrt(alkalinity): True
Real root >= 0: True
Imaginary root ~= 0: True
*** VALID ROOT FOUND ***
CO3 = 5.6478965842766

In [34]:
# Calculate CO3
CO3 = CO3*CO3
print(f"CO3 = {CO3}")

CO3 = 31.898735826684238


In [35]:
# Calculate HC03
HCO3 = np.sqrt(CO3*CO2aq)*const_A
print(f"HCO3 = {HCO3}")

HCO3 = 1208.962180607544


In [36]:
# Calculate pH
pH = -1*np.log10(np.sqrt(CO2aq/CO3*K1K2product))
print(f"pH = {pH}")

pH = 7.776915273727671


In [37]:
# Calculate Omega_A 
Omega_A = CO3*(1000*calcium)/(Kar*1e12)
print(f"Omega A = {Omega_A}")

Omega A = 0.5074737190738144


In [38]:
# Calcuate Omega_C
Omega_C = (CO3*(calcium*1000))/(Kca*1e12)
print(f"Omega C = {Omega_C}")

Omega C = 0.7872049818772241


In [39]:
# calculate Alkalinity
TCO2 = CO2aq+CO3+HCO3

In [40]:
# Summarize Output
print(f"*** {calculation_pair} SUMMARY ***")
print(f"CO3 = {CO3:.4f}")
print(f"HCO3 = {HCO3:.2f}")
print(f"CO2aq = {CO2aq:.4f}")
print(f"pCO2 = {pCO2:.2f}")
print(f"pH = {pH:.5f}")
print(f"Omega_A = {Omega_A:.5f}")
print(f"Omega_C = {Omega_C:.5f}")
print(f"alkalinity = {alkalinity:.2f}")
print(f"TCO2 = { TCO2:.2f}") 

*** pCO2, Alkalinity SUMMARY ***
CO3 = 31.8987
HCO3 = 1208.96
CO2aq = 23.5859
pCO2 = 450.00
pH = 7.77692
Omega_A = 0.50747
Omega_C = 0.78720
alkalinity = 1298.00
TCO2 = 1264.45


# MAKE A SCRIPT FOR CALCULATING FROM ALK,pCO2

In [41]:
## DEFINE THE INPUTS
calculation_pair = "pCO2, Alkalinity"
temperature = 6
salinity = 28
pCO2 = 450
alkalinity = 1298

## DEFINE THE OUTPUTS
# TCO2 calculated here
# pH calculated here
# Omega_aragonite not used or calculated
# print( f"*** Calculation pair: {calculation_pair} ***")

### Calculate all the constants

In [42]:
# CALCULATE ANCILLIARY CONSTANTS
calcium = cc.calculate_calcium(salinity) # preliminary calculation used for all cases
print( f"Calcium = {calcium:.5f}")

total_boron = cc.calculate_total_boron(salinity) # preliminary calculation used for all cases
print( f"Total Boron = {total_boron:.3f}")

Calcium = 8.60000
Total Boron = 339.360


In [ ]:
# Thermodynamic constants (equilibrium constants)
Kca = cc.calculate_Kca(salinity, temperature)
Kar = cc.calculate_Kar(salinity, temperature)
Kh = cc.calculate_Kh(salinity, temperature)
K1 = cc.calculate_K1(salinity, temperature)
K2 = cc.calculate_K2(salinity, temperature)
Kb = cc.calculate_Kb(salinity, temperature)
Kw = cc.calculate_Kw(salinity, temperature)
Kb12 = cc.calculate_Kb12(Kb, K1, K2)
K1K2quotient = K1/K2
K1K2product = K1*K2
Kw12 = cc.calculate_Kw12(Kw, K1, K2)

# Ancilliary constants (CALCULATED ELSEWHERE)
density = cc.seawater_density(salinity, temperature)
CO2aq = 23.5859
Hplus = 1

